# Retraining CLIP

## Fine Tuning Changes

1. Updated learning rate from 1e-4 to 1e-3.

  This data had a lot of very similar/hard-to-distinguish classes. The previous learning rate wasn't able to capture the fine-tune differences quickly enough. Increasing the learning rate gave the MLP more powerful predictions. Overfitting was carefully monitored and did not present an issue.

2. Changed the weighting of the logits from:

  ```
  blended_logits = 0.5 * classifier_logits + 0.5 * clip_logits
  ```
  to:
  ```
  blended_logits = 0.9 * classifier_logits + 0.1 * clip_logits
  ```

  The CLIP model was overfitting and getting bottle-necked in the similar phrasing across labels (i.e. Acura RL Sedan 2012 vs Acura TL Sedan 2012). Relying more heavily on the classifer allows noise-free classifier to do the bulk of the work, while still allowing the CLIP contrastive learning to guide the classifier slightly when it's unsure.  

## Set Up

Load in all our packages

In [ ]:
# Install necessary packages
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install scipy

import clip
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import packaging
import kagglehub
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ljm_6v_d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ljm_6v_d
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Check PyTorch version
# Ensure compatibility with CUDA
version = packaging.version.parse(torch.__version__)
if version > packaging.version.parse('1.7.0'):
    print("Pytorch version is above 1.7.0")
    print("It is version:", version)
else:
    print("PyTorch version is not above 1.7.0. Please Upgrade")

Pytorch version is above 1.7.0
It is version: 2.6.0+cu124


Get the Clip Model

In [ ]:
# Load CLIP model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)
model = model.float()

100%|███████████████████████████████████████| 338M/338M [00:24<00:00, 14.5MiB/s]


### Unfreeze more layers from CLIP

By default, many pre-trained models like CLIP freeze their internal layers. This means the weights of those layers don't get updated during training. Freezing maintains the extracted features from the initial training. But if we want the model to adapt to our new data, we need to "unfreeze" certain layers so they can be trained.

In [ ]:
for name, param in model.named_parameters():
    # This loop goes through every parameter (weight/bias) in the CLIP model.
    # `name` is a string describing which layer the parameter belongs to.
    # `param` is the actual parameter tensor (a PyTorch object containing weights).

    if "visual" in name:
        # Only unfreeze layers in the "visual" part of the model.
        # CLIP has two main parts: a visual encoder (for images) and a text encoder (for text).
        # We only want to modify the visual encoder.

        param.requires_grad = True
        # This tells PyTorch: "Yes, this parameter should be updated during training."
        # Any parameter with `requires_grad = False` will be ignored during backpropagation.


###  Define linear classification head

This is a very simple MLP neural network: a single fully connected linear layer. It's used to map the output of CLIP's image encoder to a set of class predictions. Think of it like the final decision layer that says: "I think this image is class X."

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        # Constructor for the class. Called when we create an instance of LinearClassifier.
        # `input_dim` is the size of the input features (from the CLIP image encoder: 512).
        # `num_classes` is the number of categories we want to classify

        super(LinearClassifier, self).__init__()

        self.fc = nn.Linear(input_dim, num_classes)
        # This creates the linear (fully connected) layer.
        # It takes a vector of size `input_dim`
        # and outputs a vector of size `num_classes` with values
        # representing the similarity of an image to each class.

    def forward(self, image_features):
        # This function defines how the data flows through the model during forward propogation.
        # It's called automatically during training and inference.

        return self.fc(image_features)
        # The output is a set of raw scores (logits) for each class.


## **DATA: This is the part you edit**

To run this re-training procedure this is the **only** part you want to edit. All necessary changes can be made here. Changes elsewhere may effect the model and make them difficult to compare.

### Ok, now we actually do this on the cars data set

Load data

In [ ]:
# Download latest version
path = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
print("Path to dataset files:", path)

100%|██████████| 1.83G/1.83G [01:23<00:00, 23.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2


Split out validation set

In [ ]:
dataset_root = os.path.join(path, 'car_data', 'car_data', 'train')  # Path to your "train" folder, change as needed
val_root = "/kaggle/working/cars/val"  # Path to save the validation split (working directory), change as needed
# Define paths


if not os.path.exists(val_root):
    os.makedirs(val_root)
# Create the validation root folder if it doesn't exist


for class_name in os.listdir(dataset_root):
# Split data within each class folder

    class_folder = os.path.join(dataset_root, class_name)

    if os.path.isdir(class_folder):

        image_files = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
        # Get list of image files in the class folder

        train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
        # Split the images into training and validation sets

        val_class_folder = os.path.join(val_root, class_name)
        if not os.path.exists(val_class_folder):
            os.makedirs(val_class_folder)
        # Create corresponding folders in the validation directory

        for val_image in val_files:
            src = os.path.join(class_folder, val_image)
            dst = os.path.join(val_class_folder, val_image)
            shutil.copy(src, dst)  # Use copy instead of move
        # Copy validation images to the validation folder

# After this, the validation set should be created in /kaggle/working/dogs/val


Preprocess

In [ ]:
train_transform = preprocess
val_transform = preprocess
# Define the transformation for CLIP preprocessing (same as when we loaded the model)
# CLIP preprocess automatically resizes, normalizes, and converts to tensor

train_dataset = datasets.ImageFolder(root=dataset_root, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_root, transform=val_transform)
# Create datasets for train and validation using ImageFolder

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Create DataLoaders for train and validation sets

Get classes

In [ ]:
class_names = train_dataset.classes
print(class_names)
# Extract class names from folders

with torch.no_grad():
    all_text_prompts = [f"A photo of a {classname}" for classname in class_names]
    tokenized_texts = clip.tokenize(all_text_prompts).to(device)
    text_features_all = model.encode_text(tokenized_texts)
    text_features_all = F.normalize(text_features_all, dim=-1).float()  # <- add .float() here
# Update class names with text prompt

['AM General Hummer SUV 2000', 'Acura Integra Type R 2001', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi A5 Coupe 2012', 'Audi R8 Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi S4 Sedan 2007', 'Audi S4 Sedan 2012', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S6 Sedan 2011', 'Audi TT Hatchback 2011', 'Audi TT RS Coupe 2012', 'Audi TTS Coupe 2012', 'Audi V8 Sedan 1994', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW Z4 Convertible 2012

Get test set set up for later

In [ ]:
# Paths
test_root = os.path.join(path, 'car_data', 'car_data', 'test') # Change this for your own data

# Load test set
test_dataset = datasets.ImageFolder(root=test_root, transform=preprocess)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Get all class names
class_names_test = test_dataset.classes
print("Class names:", class_names_test)

# Generate text features for all classes once
with torch.no_grad():
    all_texts = [f"A photo of a {classname}" for classname in class_names_test] # Feel free to change the prompt is desired
    tokenized_texts = clip.tokenize(all_texts).to(device)
    text_features_all = model.encode_text(tokenized_texts)  # Shape: (num_classes, 512)

Class names: ['AM General Hummer SUV 2000', 'Acura Integra Type R 2001', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi A5 Coupe 2012', 'Audi R8 Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi S4 Sedan 2007', 'Audi S4 Sedan 2012', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S6 Sedan 2011', 'Audi TT Hatchback 2011', 'Audi TT RS Coupe 2012', 'Audi TTS Coupe 2012', 'Audi V8 Sedan 1994', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW Z4 Con

## **OK, STOP EDITING HERE**

The rest of this file should work just fine without edits if you didn't change any variable names

## Let's Retrain

In [ ]:
classifier = LinearClassifier(input_dim=512, num_classes=len(class_names)).to(device)
# Initializes the classifier we defined earlier

optimizer = torch.optim.AdamW([
    {"params": model.visual.parameters(), "lr": 1e-6},
    {"params": classifier.parameters(), "lr": 1e-3}
], weight_decay=1e-4)
# We're training two parts:
# 1) model.visual: The vision encoder from CLIP — we fine-tune it very gently using a small learning rate (1e-6)
# 2) classifier: Our new linear layer — it starts from scratch, so we train it more aggressively (1e-4)
# AdamW is a common optimizer

criterion = nn.CrossEntropyLoss()
# Cross-entropy compares the predicted scores (logits) against the true label and penalizes wrong guesses.

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
# This slowly reduces the learning rate over time in a smooth cosine curve
# this is a common trick to make training more stable and avoid overshooting the minimum loss.


num_epochs = 10
# how many times we loop through the whole dataset

best_val_acc = 0
# keeps track of the best accuracy we've seen so far

patience = 3
# For early stopping — we stop training if validation accuracy doesn’t improve for 3 straight epochs
# This trains more efficiently and prevents overfitting

epochs_no_improve = 0
# how many times we've failed to beat our best accuracy

for epoch in range(num_epochs):

    classifier.train()
    # classifier.train() puts the model in training mode

    total_loss, correct, total = 0, 0, 0

    # Training ##################################################

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)"):

        images, labels = images.to(device), labels.to(device)
        image_features = model.encode_image(images).float()
        # Use CLIP’s vision model to encode the images into 512-dimension feature vectors
        image_features = F.normalize(image_features, dim=-1)
        # Normalize them (unit length) so comparisons (dot products) behave like cosine similarity

        with torch.no_grad():
            clip_logits = image_features @ text_features_all.T  # (B, num_classes)
        # Dot product between image and text features. Gives similarity scores
        # (logits) between each image and all class names.

        classifier_logits = classifier(image_features)
        # our classifier’s own guess — based on its trained weights

        clip_logits = clip_logits / clip_logits.norm(dim=-1, keepdim=True)
        classifier_logits = classifier_logits / classifier_logits.norm(dim=-1, keepdim=True)
        # Normalize - ensures the same scale

        blended_logits = 0.9 * classifier_logits + 0.1 * clip_logits
        # average the scores from CLIP and our linear classifier

        loss = criterion(blended_logits, labels)
        # Calculate the loss from the blended prediction

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Clear old gradients, backpropagate new ones, and take an optimizer step

        total_loss += loss.item()
        correct += (blended_logits.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
        # Count how many predictions were correct and update total loss and accuracy

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}, Train Acc = {train_acc:.2f}%")

    # Validation ################################################

    classifier.eval()
    # Switch model to evaluation mode

    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Val)"):
            images, labels = images.to(device), labels.to(device)
            image_features = model.encode_image(images).float()
            image_features = F.normalize(image_features, dim=-1)

            with torch.no_grad():
                clip_logits = image_features @ text_features_all.T
            classifier_logits = classifier(image_features)
            clip_logits = clip_logits / clip_logits.norm(dim=-1, keepdim=True)
            classifier_logits = classifier_logits / classifier_logits.norm(dim=-1, keepdim=True)
            blended_logits = 0.9 * classifier_logits + 0.1 * clip_logits
            correct += (blended_logits.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")
    # Count correct predictions to compute validation accuracy

    # Early Stopping #############################################

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save(classifier.state_dict(), 'best_linear_classifier.pth')
        print("Improved validation accuracy. Saved model.")
    # If we beat our best validation accuracy, save the model

    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping.")
            break
    # If we’ve gone patience epochs with no improvement, stop training early

    scheduler.step()
    # Move along the cosine schedule — lower the learning rate a bit


Epoch 1/10 (Train): 100%|██████████| 255/255 [01:59<00:00,  2.13it/s]


Epoch 1: Train Loss = 1287.9661, Train Acc = 39.91%


Epoch 1/10 (Val): 100%|██████████| 54/54 [00:20<00:00,  2.69it/s]


Epoch 1: Val Acc = 71.74%
Improved validation accuracy. Saved model.


Epoch 2/10 (Train): 100%|██████████| 255/255 [02:00<00:00,  2.12it/s]


...

Epoch 9: Val Acc = 96.65%
Improved validation accuracy. Saved model.


Epoch 10/10 (Train): 100%|██████████| 255/255 [01:59<00:00,  2.13it/s]


Epoch 10: Train Loss = 1178.3890, Train Acc = 96.70%


Epoch 10/10 (Val): 100%|██████████| 54/54 [00:20<00:00,  2.65it/s]

Epoch 10: Val Acc = 96.71%
Improved validation accuracy. Saved model.


## Compute Accuracy with Newly Trained Model

In [ ]:
def compute_topk_accuracy(logits, labels, topk=(1, 3, 5)):
    max_k = max(topk)
    batch_size = labels.size(0)

    _, pred = logits.topk(max_k, dim=1, largest=True, sorted=True)
    pred = pred.t()
    correct = pred.eq(labels.view(1, -1).expand_as(pred))

    topk_accs = {}
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        topk_accs[f"top{k}"] = (correct_k / batch_size).item() * 100.0

    return topk_accs

# Evaluate fine-tuned classifier
classifier.eval()
top1_total, top3_total, top5_total, total_samples = 0, 0, 0, 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating Fine-tuned Classifier"):
        images, labels = images.to(device), labels.to(device)
        image_features = model.encode_image(images).float()
        image_features = F.normalize(image_features, dim=-1)

        logits = classifier(image_features)
        accs = compute_topk_accuracy(logits, labels)

        top1_total += accs['top1'] * images.size(0)
        top3_total += accs['top3'] * images.size(0)
        top5_total += accs['top5'] * images.size(0)
        total_samples += images.size(0)

print(f"\nFine-tuned Classifier Accuracy:")
print(f"Top-1: {top1_total / total_samples:.2f}%")
print(f"Top-3: {top3_total / total_samples:.2f}%")
print(f"Top-5: {top5_total / total_samples:.2f}%")


Evaluating Fine-tuned Classifier: 100%|██████████| 252/252 [01:30<00:00,  2.77it/s]


Fine-tuned Classifier Accuracy:
Top-1: 81.05%
Top-3: 94.63%
Top-5: 96.80%


## Compare To Zero Shot Accuracy

In [ ]:
original_model = clip.load("ViT-B/32", device=device)[0].float().eval()
with torch.no_grad():
    tokenized_texts = clip.tokenize([f"A photo of a {classname}" for classname in class_names_test]).to(device)
    text_features_all = original_model.encode_text(tokenized_texts)
    text_features_all = F.normalize(text_features_all, dim=-1).float()

def compute_zero_shot_topk_accuracy(model, image_loader, text_features_all, device):
    model.eval()
    text_features_all = F.normalize(text_features_all, dim=-1)

    top1_total, top3_total, top5_total, total_samples = 0, 0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(image_loader, desc="Evaluating Zero-Shot CLIP"):
            images, labels = images.to(device), labels.to(device)
            image_features = model.encode_image(images).float()
            image_features = F.normalize(image_features, dim=-1)

            logits = image_features @ text_features_all.T
            accs = compute_topk_accuracy(logits, labels)

            top1_total += accs['top1'] * images.size(0)
            top3_total += accs['top3'] * images.size(0)
            top5_total += accs['top5'] * images.size(0)
            total_samples += images.size(0)

    return {
        'top1': top1_total / total_samples,
        'top3': top3_total / total_samples,
        'top5': top5_total / total_samples,
    }

# Run zero-shot evaluation
zero_shot_results = compute_zero_shot_topk_accuracy(original_model, test_loader, text_features_all, device)

print("\nZero-Shot CLIP Accuracy:")
print(f"Top-1: {zero_shot_results['top1']:.2f}%")
print(f"Top-3: {zero_shot_results['top3']:.2f}%")
print(f"Top-5: {zero_shot_results['top5']:.2f}%")

Evaluating Zero-Shot CLIP: 100%|██████████| 252/252 [01:30<00:00,  2.78it/s]


Zero-Shot CLIP Accuracy:
Top-1: 58.90%
Top-3: 82.05%
Top-5: 90.03%
